In [1]:
import pandas as pd
import numpy as np
import os
import pandas as pd
from PIL import Image
from tqdm import tqdm
import pandas as pd
import torch
from transformers import BertTokenizer, BertModel


In [2]:
from transformers import RobertaModel, RobertaTokenizer

# Load the model
model_name = "roberta-large"
model = RobertaModel.from_pretrained(model_name)

# Load the tokenizer
tokenizer = RobertaTokenizer.from_pretrained(model_name)


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [3]:
df = pd.read_csv('../Data/wiki-labeled.csv')


In [4]:
df.head()

,title,label,text,word_count
0,James Scobie,0,James Scobie (29 November 1826 – 7 October 185...,175
1,James Scobie,1,James Scobie (29 November 1826 – 7 October 189...,242
2,Dagliç sheep,0,The Dagliç is a breed of sheep found primarily...,152
3,Dagliç sheep,1,The Dagliç is a breed of sheep that is found i...,33
4,Hamdard India,0,"Hamdard Laboratories (India), is a Unani pharm...",160


In [5]:
len(df)

300000

In [6]:
gpt = df[df['label'] == 0]
human = df[df['label'] == 1]

In [7]:
print(len(gpt))
print(len(human))

150000
150000


In [8]:
gpt = gpt.sample(n=15000)
human = human.sample(n=15000)

In [9]:
print(len(gpt))
print(len(human))

15000
15000


In [10]:
def generate_bert_embeddings(text):
    inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
        #embeddings = torch.mean(outputs.last_hidden_state, dim=1)  # Mean pooling of token embeddings
        last_hidden_states = outputs.last_hidden_state
        embeddings = last_hidden_states[:,0,:]
    return embeddings.numpy()


In [11]:
import numpy as np
def remove_outliers(embeddings):
        # Calculate the mode of the embeddings
        mode_val = np.mean(embeddings[0], axis=None)
        
        # Replace outliers with the mode value
        embeddings[np.abs(embeddings - mode_val) > 2 * np.std(embeddings)] = mode_val
        
        return embeddings
def min_max_normalize(embeddings):
    # Find the minimum and maximum values in the embeddings
    min_val = np.min(embeddings)
    max_val = np.max(embeddings)
    
    # Normalize the embeddings to range [0, 255]
    normalized_embeddings = 255 * (embeddings - min_val) / (max_val - min_val)
    
    return normalized_embeddings.astype(np.uint8)  # Convert to uint8 for integer values between 0 and 255

In [12]:
def reshape_embeddings(embeddings):
    # Reshape the embeddings to 3D array
    return embeddings.reshape(32,32)

In [13]:
gpt['embeddings'] = gpt['text'].apply(generate_bert_embeddings)
gpt['normalized_embeddings'] = gpt['embeddings'].apply(remove_outliers)
gpt['normalized_embeddings1'] = gpt['normalized_embeddings'].apply(min_max_normalize)
gpt['reshaped_embeddings'] = gpt['normalized_embeddings1'].apply(reshape_embeddings)

In [39]:
gpt_embeddings = gpt['embeddings'].to_numpy()

In [40]:

with open('/home/csgrad/sunilruf/detect_llm/sunil_code/LLM/embeddings/gpt_embeddings_wiki.npy', 'wb') as f:
    #for i in range(len(gpt['embeddings'])):
    np.save(f, gpt_embeddings)
    

In [41]:
gpt_embed = np.load('/home/csgrad/sunilruf/detect_llm/sunil_code/LLM/embeddings/gpt_embeddings_wiki.npy', allow_pickle=True)

In [59]:
gpt_embed1 = list(gpt_embed)

In [50]:
print((gpt_embed[14999]))

[[-0.16914399 -0.27530515 -0.16552505 ...  0.3751053  -0.46027824
   0.38559914]]


In [56]:
print(gpt_embed.dtype)

object


In [60]:
gpt_embed_tensor = torch.tensor(gpt_embed1)


/tmp/ipykernel_606165/654233572.py:1: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /opt/conda/conda-bld/pytorch_1682343995622/work/torch/csrc/utils/tensor_new.cpp:245.)
  gpt_embed_tensor = torch.tensor(gpt_embed1)


In [63]:
print(gpt_embed_tensor[14999])

tensor([[-0.1691, -0.2753, -0.1655,  ...,  0.3751, -0.4603,  0.3856]])


In [51]:
print(gpt['embeddings'].iloc[14999])

[[-0.16914399 -0.27530515 -0.16552505 ...  0.3751053  -0.46027824
   0.38559914]]


In [64]:
human['embeddings'] = human['text'].apply(generate_bert_embeddings)
human['normalized_embeddings'] = human['embeddings'].apply(remove_outliers)
human['normalized_embeddings1'] = human['normalized_embeddings'].apply(min_max_normalize)
human['reshaped_embeddings'] = human['normalized_embeddings1'].apply(reshape_embeddings)

In [65]:
human_embeddings = human['embeddings'].to_numpy()

In [66]:

with open('/home/csgrad/sunilruf/detect_llm/sunil_code/LLM/embeddings/human_embeddings_wiki.npy', 'wb') as f:
    #for i in range(len(gpt['embeddings'])):
    np.save(f, human_embeddings)
    

In [36]:
print(gpt['reshaped_embeddings'].apply(type).unique())

[<class 'numpy.ndarray'>]


In [37]:
print(gpt['reshaped_embeddings'].isnull().sum())

0


In [ ]:
gpt.to_csv('../Data/gpt_wiki_embeddings1.csv', index=False)

In [ ]:
human.to_csv('../Data/human_wiki_embeddings1.csv', index=False)

In [43]:
df1 = pd.read_csv('../Data/gpt_wiki_embeddings.csv')

In [1]:
import matplotlib.pyplot as plt

for i in range(len(gpt['reshaped_embeddings'])):
    plt.imshow(gpt['reshaped_embeddings'].iloc[i], cmap='gray')  # Assuming grayscale image
    plt.axis('off')  # Hide axes
    plt.savefig('/home/csgrad/sunilruf/detect_llm/sunil_code/LLM/output/Research_GPT/'+str(i)+'.png', bbox_inches='tight', pad_inches=0)
    plt.close()


NameError: name 'gpt' is not defined

In [57]:
import matplotlib.pyplot as plt

for i in range(len(human['reshaped_embeddings'])):
    plt.imshow(human['reshaped_embeddings'].iloc[i], cmap='gray')  # Assuming grayscale image
    plt.axis('off')  # Hide axes
    plt.savefig('/home/csgrad/sunilruf/detect_llm/sunil_code/LLM/output/Human2/'+str(i)+'.png', bbox_inches='tight', pad_inches=0)
    plt.close()


In [74]:
gpt_text = """" ﻿The argument states that restoring the time spent on weather and local news to its formal levels, would attract viewers and avoid losing any further advertising revenue. Though the argument seems reasonable at first by comparing information on a superficial basis, it does fail to account for a lot of factors and relies on a certain number of assumptions. However, proving these assumptions faulty might result in making this argument meaningless.

Firstly, it is clear from the argument that the programs are intended to show late at night. The number of viewers at night is most probably be less than during the day. The details of the number of viewers who are focused on weather and local news, late at night have not been discussed, which in turn could not give us at least an approximate amount of the viewers lost. If any survey is conducted on the number and details of people watching a late-night news program, then it might help in determining the actual loss of viewers occurred due to this.
"""

In [75]:
gpt_embed = generate_bert_embeddings(gpt_text)
gpt_embed1 = remove_outliers(gpt_embed)
gpt_embed2 = min_max_normalize(gpt_embed1)
gpt_embed3 = reshape_embeddings(gpt_embed2)

In [76]:
plt.imshow(gpt_embed3, cmap='gray')  # Assuming grayscale image
plt.axis('off')  # Hide axes
plt.savefig('/home/csgrad/sunilruf/detect_llm/sunil_code/LLM/sample_human.png', bbox_inches='tight', pad_inches=0)
plt.close()